# Consignes

Intégrer le fichier USvideos.csv qui représente un ensemble de 8000 vidéos Youtube. 

Merger le fichier US_category_id.json pour récupérer le nom des catégories. Il conviendra de bien spécifier l'ID du document.


# Questions 
- 1) Récupérer toutes les vidéos de la chaîne Apple.
- 2) Compter le nombre de catégories différentes
- 3) Si vous ne l'avez pas déjà fait, découper les tags en listes et mettre à jour les tags de chacun des documents avec une requête update.
- 4) Récupérer les vidéos les plus vues.
- 5) Compter le nombre moyen de vues en fonction de la catégorie.
- 6) Récupérer les chaines Youtube avec la plus grande moyenne de likes.

In [63]:
import pandas as pd
import pymongo

In [64]:
client = pymongo.MongoClient()
database = client['exercices']
collection = database['youtube']

In [65]:
df_youtube = pd.read_csv("./data/USvideos.csv")
df_youtube.head(2)

video_id                                              title  \
0  XpVt6Z1Gjjo  1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED Y...   
1  K4wEI5zhHB0            iPhone X — Introducing iPhone X — Apple   

      channel_title  category_id  \
0  Logan Paul Vlogs           24   
1             Apple           28   

                                                tags    views   likes  \
0  logan paul vlog|logan paul|logan|paul|olympics...  4394029  320053   
1  Apple|iPhone 10|iPhone Ten|iPhone|Portrait Lig...  7860119  185853   

   dislikes  comment_total                                  thumbnail_link  \
0      5931          46245  https://i.ytimg.com/vi/XpVt6Z1Gjjo/default.jpg   
1     26679              0  https://i.ytimg.com/vi/K4wEI5zhHB0/default.jpg   

    date  
0  13.09  
1  13.09

## Question 0

### Netoyer les données

In [66]:
df_youtube['date'] = pd.to_datetime(df_youtube['date'], errors='coerce')
df_youtube = df_youtube.dropna(subset=['date'])
 

df_youtube.rename(columns={'video_id':'_id'}, inplace=True)
 
df_youtube = df_youtube.drop_duplicates(subset='_id', keep='last')
 
l_d =df_youtube.to_dict(orient='records')
 
import json
with open('./data/US_category_id.json', 'r') as f:
    categ = json.load(f)
 
for video in l_d:
    for cat in categ['items']:
        if video['category_id'] == int(cat['id']):
            video['category_name'] = cat['snippet']['title']

### Importer les données

In [67]:
from pymongo import MongoClient
client = MongoClient("mongo:27017")

In [68]:
db_yt = client.yt
collection_linda_yt = db_yt['videos']

In [69]:
collection_linda_yt.delete_many({})
collection_linda_yt.insert_many(l_d)

## Question 1

In [70]:
req = collection_linda_yt.find({"channel_title":"Apple"})
next(req)

{'_id': 'K4wEI5zhHB0',
 'title': 'iPhone X — Introducing iPhone X — Apple',
 'channel_title': 'Apple',
 'category_id': 28,
 'tags': 'Apple|iPhone 10|iPhone Ten|iPhone|Portrait Lighting|A11 Bionic|augmented reality|emoji|animoji|Face ID|Apple Pay|camera|smartphone',
 'views': 19707391,
 'likes': 381919,
 'dislikes': 69465,
 'comment_total': 0,
 'thumbnail_link': 'https://i.ytimg.com/vi/K4wEI5zhHB0/default.jpg',
 'date': datetime.datetime(1970, 1, 1, 0, 0),
 'category_name': 'Science & Technology'}

## Question 2

In [71]:
request = collection_linda_yt.aggregate([{ "$group": { "_id": "$category_id"}  },{ "$group": { "_id": "category_id", "count": { "$sum": 1 } } } ])
list(request) 

[{'_id': 'category_id', 'count': 16}]

## Question 3

## Question 4

In [ ]:
request = collection_linda_yt.aggregate([{ "$group": { "_id": "$category_id"}  },{ "$group": { "_id": "category_id", "count": { "$sum": 1 } } } ])
list(request)

## Question 5

## Question 6 